In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline




/home/alice/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-14 16:48:16.727154: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-14 16:48:17.353612: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
sentences = ["Quando mi ha detto che il gatto era morto  mi sono messa a pianegere", "Oggi è una bellissima giornata", "Mi fai troppo arrabbiare quando mi dici così "]

In [10]:

tokenizer = AutoTokenizer.from_pretrained("aiknowyou/it-emotion-analyzer")
model = AutoModelForSequenceClassification.from_pretrained("aiknowyou/it-emotion-analyzer")

emotion_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
emotion_analysis(sentences)

[{'label': '0', 'score': 0.778736412525177},
 {'label': '1', 'score': 0.7228806614875793},
 {'label': '3', 'score': 0.8531340956687927}]


    0: sadness
    1: joy
    2: love
    3: anger
    4: fear
    5: surprise


In [3]:


tokenizer = AutoTokenizer.from_pretrained('MilaNLProc/feel-it-italian-emotion')
model = AutoModelForSequenceClassification.from_pretrained('MilaNLProc/feel-it-italian-emotion')

emotion_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
out=emotion_analysis(sentences[0],top_k=4)

In [4]:
out

[{'label': 'sadness', 'score': 0.9951210618019104},
 {'label': 'fear', 'score': 0.003921428229659796},
 {'label': 'joy', 'score': 0.0007393912528641522},
 {'label': 'anger', 'score': 0.00021813683270011097}]

In [7]:
for o in out:
    print(o["label"])

sadness
fear
joy
anger


In [1]:
#run the planner
import os
import subprocess
def planning(command,domain_path,plan_file):
    #plan the first time and get the list of action
    os.chdir (domain_path)
    #run the planner
    fd_process = subprocess.Popen([command], stdout=subprocess.PIPE, shell=True)
    try:
        (out, err) = fd_process.communicate()
        f = open(plan_file, "w")
        f.write(out.decode())
        f.close()
        fd_exit = fd_process.returncode
        #print(fd_exit)
        return fd_exit
    except:
        print("exrrorrr")

In [20]:
command="./ff -p /home/alice/catkin_ws/src/PROPER_Sofar/proper_lpg/ -o goal1_domain.pddl -f prova_problem.pddl"
domain_path="/home/alice/catkin_ws/src/PROPER_Sofar/proper_lpg"

In [21]:
os.chdir (domain_path)
#run the planner
fd_process = subprocess.Popen([command], stdout=subprocess.PIPE, shell=True)

In [22]:
(out, err) = fd_process.communicate()


In [24]:
type(a)

str

In [25]:
f = open("plan.pddl", "w")
f.write(a)
f.close()

In [26]:
fd_exit = fd_process.returncode
#print(fd_exit)
fd_exit

0

In [32]:
output_path="/home/alice/catkin_ws/src/PROPER_Sofar/proper_lpg/plan.pddl"

In [43]:
if os.path.isfile(output_path):
    with open(output_path, "r") as plan_file:
        raw_plan = plan_file.readlines()
    plan=[]
    start=False
    for p in raw_plan:
        if "step" in p:
             start=True

        if "time spent" in p:
             start=False
        if start:
            if p[0] !=":":
                plan.append(p)
    actions_to_execute=[]
    for i in plan:
        a=i[i.find(":")+1:i.find("\n")].replace(" ","")
        if a!="":
            actions_to_execute.append(a)   
    return actions_to_execute 

else:
        print("Plan not found")

In [46]:
plan

['step    0: GREETINGS\n',
 '        1: INTRO_ACTION\n',
 '        2: FEELINGS_SKIP\n',
 '        3: CHECK_FINISH\n',
 '     \n',
 '\n']

In [55]:
actions_to_execute

['GREETINGS', 'INTRO_ACTION', 'FEELINGS_SKIP', 'CHECK_FINISH']

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from proper_lpg.load_ontology import *

In [15]:
import rospy

In [3]:
#define the actual personality
traits=["Extrovert","Introvert","Conscientious","Unscrupulous","Agreeable","Disagreeable"]
traits_preds=["(extro)","(intro)","(consc)","(unsc)","(agree)","(disagree)"]
we=0
wi=0.5
wc=0
wu=0
wa=0
wd=0
sum_weights=we +wi +wc + wu + wa + wd
weights=[we/sum_weights,wi/sum_weights,wc/sum_weights,wu/sum_weights,wa/sum_weights,wd/sum_weights]
gamma=1
perception=""
new_perception=False

In [4]:
domain_path='/home/alice/catkin_ws/src/PROPER_Sofar/proper_lpg/domain_prova.pddl'
problem_path='/home/alice/catkin_ws/src/PROPER_Sofar/proper_lpg/prova_problem.pddl'
init_pb='/home/alice/catkin_ws/src/PROPER_Sofar/proper_lpg/init_problem.pddl'

In [5]:
rospy.loginfo('Executing state INIT')
rospy.loginfo('copy pb in the correct file')        
with open(init_pb,'r') as firstfile, open(problem_path,'w') as secondfile:
    for line in firstfile:
    
        if "extroversion_coefficient" in line:
            if we!=0:
                l="        (= (extroversion_coefficient) "+str(gamma*(we/sum_weights)) +")\n"
                secondfile.write(l)
                p="        "+traits_preds[0]+"\n"
                secondfile.write(p)
            else:
                l="        (= (extroversion_coefficient) "+str(gamma*(wi/sum_weights)) +")\n"
                secondfile.write(l)
                p="        "+traits_preds[1]+"\n"
                secondfile.write(p)
        elif "conscientious_coefficient" in line:
            if wc!=0:
                l="        (= (conscientious_coefficient) "+str(gamma*(wc/sum_weights)) +")\n"
                secondfile.write(l)
                p="        "+traits_preds[2]+"\n"
                secondfile.write(p)
            else:
                l="        (= (conscientious_coefficient) "+str(gamma*(wu/sum_weights)) +")\n"
                secondfile.write(l)
                p="        "+traits_preds[3]+"\n"
                secondfile.write(p)
        elif "agreeableness_coefficient" in line:
            if wa!=0:
                l="        (= (agreeableness_coefficient) "+str(gamma*(wa/sum_weights)) +")\n"
                secondfile.write(l)
                p="        "+traits_preds[4]+"\n"
                secondfile.write(p)
            else:
                l="        (= (agreeableness_coefficient) "+str(gamma*(wd/sum_weights)) +")\n"
                secondfile.write(l)
                p="        "+traits_preds[5]+"\n"
                secondfile.write(p)
        else:
            secondfile.write(line)


In [6]:
rospy.loginfo('Reading domain and populate ontology')
populate_ontology(domain_path)


first ['(:requirements :adl :strips :typing :conditional-effects :negative-preconditions :equality :fluents s :duration-inequalities :continuous-effects :time)', 'EXTRO_ACTION', 'INTRO_ACTION', 'CONSC_ACTION', 'UNSC_ACTION', 'AGREE_ACTION', 'DISAGREE_ACTION', 'NO_REACT_SAD_EMOTION', 'REACT_SAD_EMOTION_NEG', 'NO_REACT_ANGER_EMOTION', 'REACT_ANGER_EMOTION_POS', 'REACT_ANGER_EMOTION_NEG', 'NO_REACT_HAPPY_EMOTION', 'REACT_HAPPY_EMOTION_NEG', 'REACT_HAPPY_EMOTION_AGREE', 'REACT_HAPPY_EMOTION_EXTRO', 'NO_REACT_NEUTRAL_EMOTION', 'REACT_NEUTRAL_EMOTION_POS', 'REACT_NEUTRAL_EMOTION_NEG', 'NO_REACT_TOUCH', 'APPROACH_TOUCH_AGREE', 'APPROACH_TOUCH_EXTRO', 'AVOID_TOUCH_INTRO', 'AVOID_TOUCH_DISAGREE', 'GREETINGS', 'FEELINGS', 'CHECK_FINISH']
second ['EXTRO_ACTION', 'INTRO_ACTION', 'CONSC_ACTION', 'UNSC_ACTION', 'AGREE_ACTION', 'DISAGREE_ACTION', 'NO_REACT_SAD_EMOTION', 'REACT_SAD_EMOTION_NEG', 'NO_REACT_ANGER_EMOTION', 'REACT_ANGER_EMOTION_POS', 'REACT_ANGER_EMOTION_NEG', 'NO_REACT_HAPPY_EMOTION', '

In [72]:
rospy.loginfo('Initialize function and predicates in the ontology')
initialize_functions_predicates()
rospy.loginfo('Read the problem and set the initial values of predicates and functions')
read_the_problem(problem_path) 

In [3]:
domain='/home/alice/catkin_ws/src/PROPER_Sofar/proper_lpg/goal1_domain.pddl'

In [4]:
if os.path.isfile(domain):
    with open(domain, "r") as domain_file:
        raw_domain = domain_file.readlines()

In [5]:
#ACTIONS
last_action=""
for p in raw_domain:
    if (":durative-action" in p) or (":action" in p):
        if (":durative-action" in p):
            last_action=p.replace(":durative-action","").replace("( ","").replace("\n","")
        elif (":action" in p):
            last_action=p.replace(":action","").replace("( ","").replace("\n","")
        actions.append(last_action)

for a in actions:
        actions_objects[a]=Actions(a)

In [6]:
#TYPES
t=False
for p in raw_domain:
        if (")" in p):
            t=False
        if t==True:
            type.append(p.replace("\t","").replace("\n",""))
        if (":types" in p):
            t=True
for t in type:
    types_objects[t]=Types(t)

In [7]:
#PREDICATES
t=False
for p in raw_domain:
        if (":action" in p):
            t=False
        if t==True:
            if "?" in p:
                new=p[p.find("(")+1:p.find("?")].replace(" ","")
                
            else:
                new=p[p.find("(")+1:p.find(")")].replace(" ","")
                
            if new!="":   
                predicates.append(new)
        if (":predicates" in p):
            t=True

for t in predicates:
    predicates_objects[t]=Predicates(t)

In [2]:
predicates

[]

In [8]:
#FUNCTIONS
t=False
for p in raw_domain:
        if (":predicates" in p):
            t=False
        if t==True:
            if "?" in p:
                new=p[p.find("(")+1:p.find("?")]
                
            else:
                new=p[p.find("(")+1:p.find(")")]
                
            if new!="":   
                functions.append(new)
        if (":functions" in p):
            t=True

for t in functions:
    function_objects[t]=Functions(t)

In [9]:

#connect all the relations between actions predicates and functions
t=False
pr=False
last_action=""
associated_types=[]
associated_actions=[]
associated_orders=[]

In [10]:






for p in raw_domain:
    print("--------------------")
    if (":duration" in p) or (":precondition" in p):
        pr=False
    if (":durative-action" in p) or (":action" in p):
        t=False
        if (":durative-action" in p):
            last_action=p.replace(":durative-action","").replace("( ","").replace("\n","")
        elif (":action" in p):
            last_action=p.replace(":action","").replace("( ","").replace("\n","")
        
        print(last_action)
    if t==True:
        if ";" not in p:
            operation=p.replace("and","").replace("at end","").replace("("," ").replace(")"," ")
            pred=operation.replace("decrease","").replace("increase","").replace("assign","").replace("not","").split()
            #operation=operation.split()
            print(operation)
            print(pred)
            if pred==[]:
                continue
            else:
                
                if str(pred[0])=="when":
                    check=str(pred[2])
                    for l in functions:
                        if check==l:
                            actions_objects[last_action].has_effect_function.append(function_objects[check])
                            ops=last_action.replace(" ","")+" "+operation
                            function_objects[check].has_operator.append(ops)
                            break
                else:
                    check=str(pred[0])
                    for l in predicates:
                        if check==l:
                            actions_objects[last_action].has_effect_predicates.append(predicates_objects[check])
                            ops=last_action.replace(" ","")+" "+operation
                            predicates_objects[check].has_operator.append(ops)
                            break
                            
                    for l in functions:
                        if check==l:
                            actions_objects[last_action].has_effect_function.append(function_objects[check])
                            ops=last_action.replace(" ","")+" "+operation
                            function_objects[check].has_operator.append(ops)
                            break
    if pr==True:
        params=p[p.index("(")+1:p.index(")")].replace("-","").split(" ")
        while 1:
                try:
                    params.remove("")
                except:
                    break
        c=0
        for i in params:
            if "?" in i:
                c+=1
                new_params.append(i)
                associated_actions.append(last_action)
                associated_orders.append(c)
            else:
                for j in range(c):
                    associated_types.append(i)
                c=0
    if ":parameters" in p:
        pr=True
    if ":effect" in p:
        t=True



--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
-------------

In [11]:
#PARAMS       
for t in new_params:
    parameters_objects[t]=Parameters(t)

#adding associated objects and actions of params
for i in range(len(new_params)):
    parameters_objects[new_params[i]].has_params_actions.append(actions_objects[associated_actions[i]])
    if len(parameters_objects[new_params[i]].has_params_types)==0:
        parameters_objects[new_params[i]].has_params_types.append(types_objects[associated_types[i]])
    parameters_objects[new_params[i]].has_order=associated_orders[i]

In [18]:
actions_objects["GREETINGS"].has_effect_function

[pp.interaction_level, pp.scrupulousness_level, pp.agreeableness_level]

In [19]:
function_objects["interaction_level"].has_operator

['EXTRO_ACTION                  increase  interaction_level  reward_e  \n',
 'INTRO_ACTION                       increase  interaction_level  reward_e                   \n',
 'REACT_HAPPY_EMOTION                     when  extro  increase  interaction_level  * extroversion_coefficient  react        \n',
 'REACT_NEUTRAL_EMOTION                     when  extro  decrease  interaction_level  * extroversion_coefficient  react    \n',
 'REACT_NEUTRAL_EMOTION                     when  intro  increase  interaction_level  * extroversion_coefficient  react    \t\n',
 'REACT_TOUCH                          when  extro  increase  interaction_level  * extroversion_coefficient  react    \n',
 'REACT_TOUCH                          when  intro  decrease  interaction_level  * extroversion_coefficient  react    \n',
 'GREETINGS                             decrease  interaction_level  * extroversion_coefficient  dur   \n',
 'FEELINGS                             when  extro  decrease  interaction_level  * e

In [23]:
a='REACT_TOUCH'

In [30]:
predicates_objects["touch_reacted"].is_grounded

True

In [29]:
#check preconditions if neobjects_objects[params[parameters_objects[o[2]].has_order-1].lower()].has_typeeded
ac=a.split(" ")[0]
params=a.split(" ")[1:]
preds=actions_objects[ac].has_effect_predicates
#prendo tutti i predicati effetto di quell'azione
for p in preds:
    ops=p.has_operator
    key = list(filter(lambda x: predicates_objects[x] == p, predicates_objects))[0]
    #per ogni predicato prendo tutte le operazioni di quel predicato
    for o in ops:
        o=o.replace("\n","").replace("\t","").split(" ")
        while 1:
            try:
                o.remove("")
            except:
                break
        #considero solo quelle che sono dell'azione che sto eseguendo
        if o[0]==ac:
            if o[1]=="not":
                p.is_grounded=False
                o.pop(1)
            else:
                p.is_grounded=True
            
            if len(o)>2 and p.is_grounded==True:
                if p.has_single_object==True:
                    p.has_object=[objects_objects[params[parameters_objects[o[2]].has_order-1].lower()]]
                else:
                    #print(p,p.has_object)
                    if objects_objects[params[parameters_objects[o[2]].has_order-1].lower()] not in p.has_object:
                        p.has_object.append(objects_objects[params[parameters_objects[o[2]].has_order-1].lower()])

In [31]:
function_objects["interaction_level"].has_value

In [33]:
predicates_objects["agree"].is_grounded

In [ ]:

funcs=actions_objects[ac].has_effect_function
#print(funcs)
if funcs!=[]:
    for f in funcs:
        ops=f.has_operator
        key = list(filter(lambda x: function_objects[x] == f, function_objects))[0]
        #per ogni predicato prendo tutte le operazioni di quel predicato
        #print(ops)
        for o in ops:
            o=o.replace("\n","").replace("\t","").split(" ")
            while 1:
                try:
                    o.remove("")
                except:
                    break
            if o[0]==ac:
                #print(o)
                #print(f,ac,f.has_value)
                if o[1]=="when":
                    if predicate_objects[o[2]].is_grounded:
                        if o[3]=="assign":
                            f.has_value=int(o[5])
                        elif o[3]=="increase":
                            actual_value=f.has_value
                            if o[5]=="*":
                                v1=function_objects[o[6]].has_value
                                if o[7]=="+":
                                    v2=function_objects[o[8]].has_value+float(o[9])
                                elif o[7]=="-":
                                    v2=function_objects[o[8]].has_value-float(o[9])
                                else:
                                    v2=function_objects[o[7]].has_value
                                f.has_value=actual_value + v1*v2
                            else:
                                try:
                                    f.has_value=actual_value + float(o[num+2])
                                except:
                                    f.has_value=actual_value + function_objects[o[3]].has_value

                        elif o[3]=="decrease":
                            actual_value=f.has_value
                            if o[5]=="*":
                                v1=function_objects[o[6]].has_value
                                if o[7]=="+":
                                    v2=function_objects[o[8]].has_value+float(o[9])
                                elif o[7]=="-":
                                    v2=function_objects[o[8]].has_value-float(o[9])
                                else:
                                    v2=function_objects[o[7]].has_value
                                f.has_value=actual_value - v1*v2
                            else:
                                try:
                                    f.has_value=actual_value - float(o[3])
                                except:
                                    f.has_value=actual_value - function_objects[o[3]].has_value
                else:
                    num=1
                if o[num]=="assign":
                    f.has_value=int(o[num+2])
                elif o[num]=="increase":
                    actual_value=f.has_value
                    if o[num+2]=="*":
                        v1=function_objects[o[num+3]].has_value
                        v2=function_objects[o[num+4]].has_value
                        f.has_value=actual_value + v1*v2
                    else:
                        try:
                            f.has_value=actual_value + float(o[num+2])
                        except:
                            f.has_value=actual_value + function_objects[o[3]].has_value
                elif o[1]=="decrease":
                    actual_value=f.has_value
                    if o[3]=="*":
                        if o[5]=="+":
                            v1=function_objects[o[4]].has_value #coeff
                            v2=function_objects[o[6]].has_value #dur
                            f.has_value=actual_value - v1*(v2+int(o[7]))
                        else:
                            v1=function_objects[o[4]].has_value
                            v2=function_objects[o[5]].has_value
                            f.has_value=actual_value - v1*v2
                    else:
                        try:
                            f.has_value=actual_value - float(o[3])
                        except:
                            f.has_value=actual_value - function_objects[o[3]].has_value
                else:
                    print("NO OPERATION FOUND")
                #print(f,ac,f.has_value)